In [1]:
import pandas as pd
import numpy as np

In [2]:
# 출동 전처리 데이터 및 날씨데이터 가져오기
df_res = pd.read_csv("data/rescue_pp.csv")
df_wea = pd.read_excel('data/충남 15개 지점 일값 자료.xlsx', sheet_name=None, engine='openpyxl')

In [3]:
# 불필요 컬럼 제거 및 이름 변경
del df_wea['서산 등 5개 지점 일값자료']
df_wea = pd.concat(df_wea.values())
df_wea = df_wea.drop(columns=['지점', '최대 순간풍속 시각(hhmi)', '최대 순간 풍속 풍향(16방위)', '최대 순간 풍속 풍향(deg)'])
df_wea = df_wea.rename(columns={
    '평균기온(°C)': 'AVG_TEMP', '최저기온(°C)': 'MIN_TEMP', '최고기온(°C)': 'MAX_TEMP', 
    '일강수량(mm)': 'DAY_PRE', '최대 순간 풍속(m/s)': 'MAX_WSPEED', '평균 풍속(m/s)': 'AVG_WSPEED'
})
df_wea.head(3)

,지점명,일시,AVG_TEMP,MIN_TEMP,MAX_TEMP,DAY_PRE,MAX_WSPEED,AVG_WSPEED
0,서산,2014-01-01,3.0,-4.6,9.3,NaN,12.6,2.5
1,서산,2014-01-02,0.7,-4.8,7.8,NaN,7.4,0.8
2,서산,2014-01-03,2.3,-4.3,9.6,NaN,5.5,1.1


In [4]:
df_res['지점명'] = df_res['EMD_NAME'].apply(lambda x: str(x).split()[1][:2])
df_res['일시'] = df_res['TIME'].apply(lambda x: str(x).split()[0])
df_res['일시'] = pd.to_datetime(df_res['일시'])
df_res.head(3)

,EMD_CD,EMD_NAME,TIME,LON,LAT,TYPE,LABEL,지점명,일시
0,44760310,충청남도 부여군 규암면,2014-08-25 07:29:00,126.884171,36.275284,호우주의보,0,부여,2014-08-25
1,44760420,충청남도 부여군 장암면,2014-08-25 10:00:00,126.886455,36.243935,호우주의보,0,부여,2014-08-25
2,44760350,충청남도 부여군 구룡면,2014-08-25 10:39:00,126.812825,36.252592,호우주의보,0,부여,2014-08-25


In [8]:
# 라벨데이터랑 merge후 결측치 확인
df_res_wea = pd.merge(df_res, df_wea, how='left', on=['지점명', '일시'])
df_res_wea['year'] = df_res_wea['일시'].dt.year
df_res_wea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6597 entries, 0 to 6596
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   EMD_CD      6597 non-null   int64         
 1   EMD_NAME    6597 non-null   object        
 2   TIME        6597 non-null   object        
 3   LON         6597 non-null   float64       
 4   LAT         6597 non-null   float64       
 5   TYPE        6597 non-null   object        
 6   LABEL       6597 non-null   int64         
 7   지점명         6597 non-null   object        
 8   일시          6597 non-null   datetime64[ns]
 9   AVG_TEMP    6460 non-null   float64       
 10  MIN_TEMP    6464 non-null   float64       
 11  MAX_TEMP    6471 non-null   float64       
 12  DAY_PRE     6415 non-null   float64       
 13  MAX_WSPEED  6430 non-null   float64       
 14  AVG_WSPEED  6413 non-null   float64       
 15  year        6597 non-null   int32         
dtypes: datetime64[ns](1), fl

## 주의할점
- 기상청 데이터에 결측치가 많음
- 호우경보나 호우주의보 특성상 해당 특보 시 강수량이나 풍속 등이 일정 수치 넘길 것
- 즉 1, 2년 사이 같은 특보의 기상정보로 평균 내는 것이 가능

In [9]:
# 결측값 처리 함수
# 결측값이 아니면서 동일년도 +1, +2 인 항목의 평균값으로 대체
def handle_na(df, col):
    for idx in list(df.loc[df[col].isna()].index):
        if pd.isna(df.loc[idx, col]):
            df.loc[idx, col] = df.loc[(
                ~df[col].isna()) & 
                (df['TYPE'] == df.loc[idx, "TYPE"]) & 
                ((df['year'] == df.loc[idx, "year"] + 1) | 
                (df['year'] == df.loc[idx, "year"] + 2) |
                (df['year'] == df.loc[idx, "year"])), col
            ].mean()

In [10]:
# 결측값 전부 처리
for col in ["AVG_TEMP", "MIN_TEMP", "MAX_TEMP", "DAY_PRE", "MAX_WSPEED", "AVG_WSPEED"]:
    handle_na(df_res_wea, col)

In [12]:
# 결측치 확인
df_res_wea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6597 entries, 0 to 6596
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   EMD_CD      6597 non-null   int64         
 1   EMD_NAME    6597 non-null   object        
 2   TIME        6597 non-null   object        
 3   LON         6597 non-null   float64       
 4   LAT         6597 non-null   float64       
 5   TYPE        6597 non-null   object        
 6   LABEL       6597 non-null   int64         
 7   지점명         6597 non-null   object        
 8   일시          6597 non-null   datetime64[ns]
 9   AVG_TEMP    6597 non-null   float64       
 10  MIN_TEMP    6597 non-null   float64       
 11  MAX_TEMP    6597 non-null   float64       
 12  DAY_PRE     6597 non-null   float64       
 13  MAX_WSPEED  6597 non-null   float64       
 14  AVG_WSPEED  6597 non-null   float64       
 15  year        6597 non-null   int32         
dtypes: datetime64[ns](1), fl

In [15]:
# 필요한 컬럼만 선별
df_res_wea = df_res_wea.loc[
    :, ['EMD_CD', 'EMD_NAME', 'LON', 'LAT', 'TIME', 'TYPE', 'AVG_TEMP', 'MIN_TEMP',
        'MAX_TEMP', 'DAY_PRE', 'MAX_WSPEED', 'AVG_WSPEED', 'LABEL']
]
df_res_wea.head(3)

,EMD_CD,EMD_NAME,LON,LAT,TIME,TYPE,AVG_TEMP,MIN_TEMP,MAX_TEMP,DAY_PRE,MAX_WSPEED,AVG_WSPEED,LABEL
0,44760310,충청남도 부여군 규암면,126.884171,36.275284,2014-08-25 07:29:00,호우주의보,23.2,21.9,25.5,34.0,5.6,1.2,0
1,44760420,충청남도 부여군 장암면,126.886455,36.243935,2014-08-25 10:00:00,호우주의보,23.2,21.9,25.5,34.0,5.6,1.2,0
2,44760350,충청남도 부여군 구룡면,126.812825,36.252592,2014-08-25 10:39:00,호우주의보,23.2,21.9,25.5,34.0,5.6,1.2,0


In [16]:
# 전처리 데이터 저장
df_res_wea.to_csv("data/rescue_weather_pp.csv", index=False)